In [30]:
import torch
import numpy as np

# check if CUDA is available
device = torch.cuda.is_available()
device

if not device:
    device = 'cpu'
    print('CUDA is not available.  Training on CPU ...')
else:
    device = 'cuda'
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [31]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [32]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

train_data = datasets.MNIST('MNIST', train=True,
                              download=True, transform=transform)
test_data = datasets.MNIST('MNIST', train=False,
                             download=True, transform=transform)

In [33]:
data_loader = torch.utils.data.DataLoader(dataset = train_data, 
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torchvision import transforms


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.dropout1 = nn.Dropout()
        
        self.fc = nn.Linear(7 * 7 * 64, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
       
        return out

In [55]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [56]:
model = CNN().to(device)

In [57]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [58]:
total_batch = len(data_loader)
# total_batch

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

RuntimeError: output with shape [1, 28, 28] doesn't match the broadcast shape [3, 28, 28]

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [32]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(1)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(1)

In [33]:
learning_rate = 0.001
epochs = 15
batch_size = 128

In [34]:
train_data = datasets.MNIST('MNIST', train = True, transform = transforms.ToTensor(), download = True)

test_data = datasets.MNIST('MNIST', train = False, transform = transforms.ToTensor(), download = True)

In [35]:
data_loader = torch.utils.data.DataLoader(dataset = train_data,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [36]:
class CNN1(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.fc = nn.Linear(7 * 7 * 64, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = nn.Flatten()(out)
        out = self.fc(out)
        
        return out

In [37]:
model = CNN1().to(device)

In [38]:
criterion = torch.nn.CrossEntropyLoss().to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [39]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 468


In [40]:
for epoch in range(epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch:{:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:   1] cost = 0.138704374
[Epoch:   2] cost = 0.0518866293
[Epoch:   3] cost = 0.0388563387
[Epoch:   4] cost = 0.0314144604
[Epoch:   5] cost = 0.024748601
[Epoch:   6] cost = 0.0217573009
[Epoch:   7] cost = 0.0168252457
[Epoch:   8] cost = 0.0160554796
[Epoch:   9] cost = 0.0123447953
[Epoch:  10] cost = 0.0119061982
[Epoch:  11] cost = 0.00735120196
[Epoch:  12] cost = 0.00792108104
[Epoch:  13] cost = 0.0105834641
[Epoch:  14] cost = 0.00560274255
[Epoch:  15] cost = 0.00395876588


In [41]:
with torch.no_grad():
    X_test = test_data.data.view(len(test_data), 1, 28, 28).float().to(device)
    Y_test = test_data.targets.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', round(accuracy.item(), 9))

Accuracy: 0.989899993
